# 마이크로 컨트롤러에서의  제스처 인식 모델 훈련

본 노트북 파일은 [TensorFlow Lite for Microcontrollers](https://tensorflow.org/lite/microcontrollers/overview) 환경에서의 20kb 제스처 인식 모델을 훈련시키는 방법을 나타내며, [magic_wand](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/lite/micro/examples/magic_wand) 예제 애플리케이션에서 사용된 것과 동일한 모델을 생성할 것이다. 

이 모델은 [Google Colaboratory](https://colab.research.google.com)에서도 사용할 수 있도록 설계되었으므로 쉽게 따라할 수 있다.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/magic_wand/train/train_magic_wand_model.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/micro/examples/magic_wand/train/train_magic_wand_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


GPU 가속을 사용하면 훨씬 빨라져 모델 훈련 시간이 단축된다. 훈련을 진행하기 전에 **런타임->런타임 유형 변경**으로 이동하여 **GPU**를 선택해 GPU 런타임을 사용하고 있는지 확인해보자. GPU로 런타임을 변경하면 훈련에 약 5분이 소요된다.

## 종속성 구성

올바른 버전의 TensorFlow가 사용되도록 다음 셀을 실행해보자.

In [0]:
%tensorflow_version 2.x


지금부터 훈련 스크립트가 포함되어 있는 TensorFlow 레파지토리를 클론한 뒤, 워크스페이스에 복제할 것이다. 

In [1]:
# 깃허브(GitHub)에서 레파지토리를 클론(Clone)
!git clone --depth 1 -q https://github.com/tensorflow/tensorflow
# 훈련 스크립트를 워크스페이스에 복사
!cp -r tensorflow/tensorflow/lite/micro/examples/magic_wand/train train

## 데이터 준비하기

다음으로 데이터를 다운로드 한 뒤, 훈련 스크립트가 위치한 폴더에 압축을 해제하도록 하자. 

In [0]:
# 훈련 데이터를 다운로드 한다.
!wget http://download.tensorflow.org/models/tflite/magic_wand/data.tar.gz
# 훈련 데이터를 적합한 위치에 압축을 해제한다. 
!tar xvzf data.tar.gz -C train 1>/dev/null

그런 다음 데이터를 훈련, 검증, 테스트 셋으로 분할하는 스크립트를 실행하자.

In [0]:
# 스크립트는 train 디렉토리 내에서 실행해야한다. 
%cd train
# 데이터 전처리 스크립트 실행
!python data_prepare.py
# 데이터를 사람을 기준으로 분할
!python data_split_person.py

## 텐서보드 로드 
이제 훈련이 진행됨에 따라 정확도와 손실률을 그래프로 표시하기 위해 TensorBoard를 설정해보자.

In [0]:
# 텐서보드 로드
%load_ext tensorboard
%tensorboard --logdir logs/scalars

## 훈련 시작
다음 셀은 훈련 과정을 시작한다. GPU 런타임에 약 5분이 소요되고, 몇 번의 에포크 후에 텐서보드에 측정 지표가 표시된다.

In [0]:
!python train.py --model CNN --person true

## C 소스 파일 생성 

`train.py` 스크립트는 모델 `model.tflite`를 훈련 스크립트 디렉토리에 생성하는데, 다음 셀에서 생성된 모델을 C++ 소스 파일로 변환한다. 변환된 C++ 소스파일은 TensorFlow Lite for Microcontrollers 환경에서 사용할 수 있다. 

In [0]:
# xxd를 사용할 수 없을 시 설치한다. 
!apt-get -qq install xxd
# 파일을 C 소스 파일로 저장한다. 
!xxd -i model.tflite > /content/model.cc
# 소스파일의 내용을 확인한다. 
!cat /content/model.cc